In [ ]:
import sys, os

os.path.join("", "classes")

abs_path = os.path.abspath("../Agente")
modules_path = os.path.join(abs_path, "modules")
classes_path = os.path.join(abs_path, "classes")
classes_path = os.path.join(abs_path, "custom_env")

sys.path.append(modules_path)
sys.path.append(classes_path)


if 'local_agent' in sys.modules:
    del sys.modules['local_agent']
import local_agent


from IPython.display import display
from stable_baselines3.common.utils import set_random_seed


if __name__ == "__main__":
    env_id = 'RedesOpticasEnv-v0'
    n_ciclos = 200

    num_ont = 16
    TxRate = 10e9
    temp_ciclo = 0.002
    B_guaranteed = 600e6

    num_timesteps = 1000
    num_tests = 1
    num_envs = 1
    algorithm = 'ppo'
    seed = 42

    agent = local_agent.LocalAgent(num_ont, TxRate, temp_ciclo, B_guaranteed, seed)
    agent.create_model(env_id, num_envs, algorithm, n_ciclos)
    agent.train_model(num_timesteps)
    agent.exec_simulation(n_ciclos, num_tests)
    agent.plot_results()